# 1. Samples pre-processing

This task will fetch the raw data from the remote database given a SQL query file.

## 1.1 Import libraries

In [1]:
import os
import sys
import yaml
import numpy as np
import pandas as pd

from pymongo import MongoClient

sys.path.append('..')

from utils import get_csv_files, memory_usage, \
correct_encoding, update_metadata

In [2]:
def save_df_page(page, chunks, collection, exclude=['id']):
    # concatenate data chunks -> careful benchmark this
    df = pd.concat(chunks, axis=0)
    
    # store page metadata in mongo
    update_metadata(collection, df)
    
    # save output to a parquet file with brotli compression
    df.to_parquet(os.path.join(model_storage, '{}.{}.parquet'.format(model, page)), compression='brotli')
    
    return page + 1

## 1.2 Load the data

In [3]:
DATA_DIR = '../../data'
STORAGE_DIR = '../../storage'

config = yaml.load(open('../config.yml'), Loader=yaml.FullLoader)

model = 'samples'
model_storage = os.path.join(STORAGE_DIR, model)

if not os.path.exists(model_storage):
    os.makedirs(model_storage)
    
client = MongoClient('mongodb://{}:{}@{}:{}'.format(config['MONGO_USERNAME'], config['MONGO_PASSWORD'],
                                                    config['MONGO_HOST'], config['MONGO_PORT']))
metadata_db = client[config['MONGO_DATABASE']]

model_metadata = metadata_db[model]

model_metadata.delete_many({})
model_metadata.insert_one({'pages': []})

In [4]:
csv_files = get_csv_files(os.path.join(DATA_DIR, 'raw', model, '*.csv'))

In [5]:
if not csv_files:
    raise FileNotFoundError('Couldn\'t find any csv files! Please make sure the filepath exists')

In [6]:
df = pd.read_csv(csv_files[0], sep=';')

# sample rows to list columns and dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298809 entries, 0 to 298808
Data columns (total 32 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    298809 non-null  int64  
 1   device_id             298809 non-null  int64  
 2   timestamp             298809 non-null  object 
 3   battery_state         298809 non-null  object 
 4   battery_level         298809 non-null  float64
 5   timezone              298809 non-null  object 
 6   country_code          298809 non-null  object 
 7   charger               298809 non-null  object 
 8   health                298809 non-null  object 
 9   voltage               298809 non-null  float64
 10  temperature           298809 non-null  float64
 11  usage                 298809 non-null  float64
 12  up_time               298809 non-null  int64  
 13  sleep_time            298809 non-null  int64  
 14  network_status        298809 non-null  object 
 15  

In [7]:
df.describe(include=[np.number])

,id,device_id,battery_level,voltage,temperature,usage,up_time,sleep_time,wifi_signal_strength,wifi_link_speed,...,roaming_enabled,bluetooth_enabled,location_enabled,power_saver_enabled,nfc_enabled,developer_mode,free,total,free_system,total_system
count,298809.000000,298809.000000,298809.000000,298809.000000,298809.000000,298809.000000,2.988090e+05,2.988090e+05,298809.000000,298809.000000,...,298809.000000,298809.000000,298809.000000,298809.000000,298809.000000,298809.000000,298809.000000,298809.000000,298809.000000,298809.000000
mean,150133.189358,982.856246,57.420627,10.067230,30.802061,0.480892,4.133925e+05,2.239833e+05,-103.654354,49.664783,...,0.025073,0.190961,0.487392,0.064081,0.155973,0.096567,9739.732598,22384.894243,295.728971,2696.538625
std,86627.615952,587.635906,25.781482,157.580041,5.004957,0.839212,6.871774e+05,4.196295e+05,375.505466,89.545422,...,0.156347,0.393059,0.499842,0.244898,0.362830,0.295367,13858.079778,18468.515310,308.322161,1057.496174
min,1.000000,1.000000,1.000000,0.000000,-5.800000,-107.000000,2.500000e+01,0.000000e+00,-9999.000000,-1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,51.000000,372.000000,0.000000,218.000000
25%,75153.000000,466.000000,37.000000,3.840000,27.900000,0.260000,5.731700e+04,2.210400e+04,-127.000000,-1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1410.000000,10766.000000,95.000000,1891.000000
50%,150212.000000,970.000000,59.000000,4.020000,31.000000,0.450000,1.877630e+05,7.996200e+04,-74.000000,39.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4199.000000,12414.000000,182.000000,2496.000000
75%,225200.000000,1467.000000,79.000000,4.180000,34.000000,0.690000,4.530230e+05,2.231340e+05,-58.000000,65.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,12554.000000,25889.000000,402.000000,3610.000000
max,300000.000000,2224.000000,100.000000,4404.580000,54.000000,339.430000,1.210760e+07,9.440295e+06,0.000000,2552.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,215812.000000,235216.000000,2754.000000,7132.000000


In [8]:
df.describe(exclude=[np.number])

,timestamp,battery_state,timezone,country_code,charger,health,network_status,mobile_data_status,mobile_data_activity,wifi_status
count,298809,298809,298809,298809,298809,298809,298809,298809,298809,298809
unique,192899,4,49,35,3,4,18,4,4,5
top,2017-10-12 20:57:22,Charging,Europe/Lisbon,pt,ac,Good,WIFI,disconnected,none,enabled
freq,11,169760,136958,249562,156367,298615,174464,238171,233461,215603


## 1.3 Basic data pre-processing

In [9]:
memory_usage(df) * 5000000 / 100

11803000.0